In [ ]:
import numpy as np
import pandas as pd
import json
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn import preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
raw_df = pd.read_csv("/content/drive/Shareddrives/CIS 522/code/input/TikTok Data/batch_1_100/fyp_1649361969234.csv")
raw_df.head()


,id,secretID,text,createTime,authorMeta.id,authorMeta.secUid,authorMeta.name,authorMeta.nickName,authorMeta.verified,authorMeta.signature,...,videoMeta.width,videoMeta.duration,diggCount,shareCount,playCount,commentCount,downloaded,mentions,hashtags,effectStickers
0,6862153058223197445,6862153058223197445,To the 🐝 🐝 🐝 #fyp,1597719521,6748458643983238149,MS4wLjABAAAAIQGNoHByQvokcLbWCGnIfas0OV2nctsZnR...,bellapoarch,Bella Poarch,True,This ain’t Build a B*tch💅🏻💕,...,576,10,56300000,1900000,684100000,2100000,True,[],"[{""id"":""229207"",""name"":""fyp"",""title"":"""",""cover...","[{""id"":""479381"",""name"":""Face Zoom""}]"
1,6911406868699073798,6911406868699073798,😳This is one of my best drawings #dlaciebie #t...,1609187323,6815820521784787973,MS4wLjABAAAAChcdIOWtt0MsKBex55u0trgweWs91WSlED...,fredziownik_art,Franek Bielak,True,"Yes, I’m from🇵🇱\nfredziownik123@gmail.com",...,540,26,49800000,1000000,275700000,834300,True,[],"[{""id"":""10638098"",""name"":""dlaciebie"",""title"":""...",[]
2,7028775404173413678,7028775404173413678,bruh this shit has me WHEEZING #fyp,1636514303,6753362178001224710,MS4wLjABAAAAUFwzAHWhUBwtDw0QEon-wwsnJ9Rw9mf6G7...,adrianchateau,Adrian Chateau Wiles,False,my dog’s name is keiko,...,576,9,40100000,2000000,233500000,547900,True,[],"[{""id"":""229207"",""name"":""fyp"",""title"":"""",""cover...",[]
3,6975140587196517638,6975140587196517638,"Fill the cheeks Squishy, fill the cheeks #fyp ...",1624026477,6798572058492339206,MS4wLjABAAAApBRCSyte5f1loiOXfgMAa_ZqJIf4rMpqha...,chipmunksoftiktok,Chipmunks of TikTok,False,The official home of Dinky-Squishy-Mooshy-Spon...,...,576,24,35200000,995200,280500000,323800,True,[],"[{""id"":""229207"",""name"":""fyp"",""title"":"""",""cover...",[]
4,6984473039605959941,6984473039605959941,#fyp #foryou #foryoupage,1626199356,6897222533029479426,MS4wLjABAAAAYLkvqNitbbG7n87qKRZH7DOvLTr9EcxBmD...,cacha.7.griselda,cachita7gri,False,Soy de Paraná Entre Rios\nsiganme @ka.chaa por...,...,576,12,31400000,2300000,265800000,1500000,True,[],"[{""id"":""229207"",""name"":""fyp"",""title"":"""",""cover...",[]


In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       100 non-null    int64  
 1   secretID                 100 non-null    int64  
 2   text                     100 non-null    object 
 3   createTime               100 non-null    int64  
 4   authorMeta.id            100 non-null    int64  
 5   authorMeta.secUid        100 non-null    object 
 6   authorMeta.name          100 non-null    object 
 7   authorMeta.nickName      100 non-null    object 
 8   authorMeta.verified      100 non-null    bool   
 9   authorMeta.signature     96 non-null     object 
 10  authorMeta.avatar        100 non-null    object 
 11  authorMeta.following     100 non-null    int64  
 12  authorMeta.fans          100 non-null    int64  
 13  authorMeta.heart         100 non-null    int64  
 14  authorMeta.video         10

In [ ]:
#keep only response variables and features
cols_to_keep = ['authorMeta.verified',
       'authorMeta.following', 'authorMeta.fans', 'authorMeta.heart',
       'authorMeta.video', 'authorMeta.digg', 'musicMeta.musicOriginal',
       'musicMeta.duration', 'videoMeta.height', 'videoMeta.width',
       'videoMeta.duration', 'diggCount', 'shareCount', 'playCount',
       'commentCount', 'mentions', 'hashtags', 'effectStickers']
df = raw_df[cols_to_keep]
df.head(1)


,authorMeta.verified,authorMeta.following,authorMeta.fans,authorMeta.heart,authorMeta.video,authorMeta.digg,musicMeta.musicOriginal,musicMeta.duration,videoMeta.height,videoMeta.width,videoMeta.duration,diggCount,shareCount,playCount,commentCount,mentions,hashtags,effectStickers
0,True,589,88800000,2100000000,454,8320,False,10,1024,576,10,56300000,1900000,684100000,2100000,[],"[{""id"":""229207"",""name"":""fyp"",""title"":"""",""cover...","[{""id"":""479381"",""name"":""Face Zoom""}]"


In [ ]:
# parse effects and hashtags
hashtags = []
effects = []
mentions = []
df['hashtags'] = df['hashtags'].apply(json.loads)
df['effectStickers'] = df['effectStickers'].apply(json.loads)
df['mentions'] = df['mentions'].apply(json.loads)
for _, row in df.iterrows():
  for hashtag in row['hashtags']:
    if hashtag not in hashtags:
      hashtags.append(hashtag)
  for effect in row['effectStickers']:
    if effect not in effects:
      effects.append(effect)
  for mention in row['mentions']:
    if mention not in mentions:
      mentions.append(mention.strip("@"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [ ]:
print(mentions)

['weeememes', 'malek', 'wearfigs', 'a', 'giulianayonemoto0', 'nameaana', 'tommy', 'shawnmendes', 'camilacabello', 'jinjoofficial', 'bboyvero', 'jinjojavelin', 'buiibeu', 'alyssa', 'gordonramsayofficial', 'lynn_azar', 'whatsondubai', 'Dasha', 'nikhagen', 'imjustbait', 'justinemf21', 'chloeboboey', 'jmko_music', 'nhedz0603', 'marblemannequin']


In [ ]:
new_hashtag_cols = [d['id'] for d in hashtags]
new_effects_cols = [d['id'] for d in effects]

In [ ]:
for col in new_hashtag_cols:
  df["hashtag_"+col] = False
for col in new_effects_cols:
  df["effect_"+col] = False
for col in mentions:
  df[col] = False

for i, row in df.iterrows():
  for hashtag in row['hashtags']:
    df.loc[i,"hashtag_"+hashtag['id']] = True
  for effect in row['effectStickers']:
    df.loc[i,"effect_"+effect['id']] = True
  for mention in row['mentions']:
    df.loc[i, mention.strip("@")] = True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragme

In [ ]:
# drop original hashtag and effects cols
df = df.drop(columns=['hashtags', 'effectStickers','mentions'])

In [ ]:
# df['mentions'] = df['mentions'].apply(json.loads)
# pd.get_dummies(df).head(1)
# df[df['mentions'].apply(lambda x: x!=[])] # shows non empty rows

In [ ]:
df.shape

(100, 351)

In [ ]:
df.iloc[:, :15].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   authorMeta.verified      100 non-null    bool 
 1   authorMeta.following     100 non-null    int64
 2   authorMeta.fans          100 non-null    int64
 3   authorMeta.heart         100 non-null    int64
 4   authorMeta.video         100 non-null    int64
 5   authorMeta.digg          100 non-null    int64
 6   musicMeta.musicOriginal  100 non-null    bool 
 7   musicMeta.duration       100 non-null    int64
 8   videoMeta.height         100 non-null    int64
 9   videoMeta.width          100 non-null    int64
 10  videoMeta.duration       100 non-null    int64
 11  diggCount                100 non-null    int64
 12  shareCount               100 non-null    int64
 13  playCount                100 non-null    int64
 14  commentCount             100 non-null    int64
dtypes: bool

In [ ]:
df.iloc[:, :15].columns

Index(['authorMeta.verified', 'authorMeta.following', 'authorMeta.fans',
       'authorMeta.heart', 'authorMeta.video', 'authorMeta.digg',
       'musicMeta.musicOriginal', 'musicMeta.duration', 'videoMeta.height',
       'videoMeta.width', 'videoMeta.duration', 'diggCount', 'shareCount',
       'playCount', 'commentCount'],
      dtype='object')

## Virality Score
Virality score is defined as average of the scaled video stats (likes, shares, plays, comments) subtracted by the number of followers for that author.

In [ ]:
add_cols = ['diggCount','shareCount','playCount','commentCount']
sub_cols = ['authorMeta.fans']
virality_df = df.copy()
virality_df['virality_score'] = virality_df[add_cols].sum(axis=1) - virality_df[sub_cols].mean(axis=1)

In [ ]:
virality_df

,authorMeta.verified,authorMeta.following,authorMeta.fans,authorMeta.heart,authorMeta.video,authorMeta.digg,musicMeta.musicOriginal,musicMeta.duration,videoMeta.height,videoMeta.width,...,whatsondubai,Dasha,nikhagen,imjustbait,justinemf21,chloeboboey,jmko_music,nhedz0603,marblemannequin,virality_score
0,True,589,88800000,2100000000,454,8320,False,10,1024,576,...,False,False,False,False,False,False,False,False,False,655600000.0
1,True,99,11800000,231900000,191,27700,False,34,960,540,...,False,False,False,False,False,False,False,False,False,315534300.0
2,False,159,985700,40400000,3,8084,True,13,1024,576,...,False,False,False,False,False,False,False,False,False,275162200.0
3,False,51,13100000,168800000,441,9318,True,24,1024,576,...,False,False,False,False,False,False,False,False,False,303919000.0
4,False,9650,2900000,38900000,1073,14900,True,11,1024,576,...,False,False,False,False,False,False,False,False,False,298100000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,433,14700000,610800000,1517,11100,True,27,1024,576,...,False,False,False,False,False,False,False,False,False,65978800.0
96,False,262,1700000,39400000,773,31800,True,48,1024,576,...,False,False,False,False,False,False,False,False,False,74083900.0
97,False,197,2900000,45900000,130,149,True,180,1024,576,...,False,False,False,False,False,False,False,False,False,283774000.0
98,False,588,403000,15600000,13,19300,False,13,1024,576,...,False,False,False,False,False,False,False,False,False,94126700.0


## Scaling

In [ ]:
cols_to_scale = ['musicMeta.duration', 'videoMeta.height',
       'videoMeta.width', 'videoMeta.duration']

# Regression Models

In [ ]:
X = virality_df.drop(columns=["diggCount", 
                     'shareCount',
                     'playCount',
                     'commentCount',
                     'authorMeta.verified',
                     "authorMeta.following",
                     'authorMeta.fans',
                     'authorMeta.heart',
                     'authorMeta.video',
                     'authorMeta.digg'])
y = virality_df[['virality_score']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
scaler_x = preprocessing.StandardScaler()

X_train[cols_to_scale] = scaler_x.fit_transform(X_train[cols_to_scale])
X_test[cols_to_scale] = scaler_x.transform(X_test[cols_to_scale])


In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_test)
print(mean_squared_error(y_pred, y_test))
print(r2_score(y_pred, y_test))


8.755958920877067e-14
1.0


In [ ]:
compare_df = raw_df.loc[y_test.index]
compare_df['virality_score_pred'] = y_pred
compare_df['virality_score'] = y_test
compare_df.iloc[1,:]
# compare_df.loc[:, add_cols+sub_cols+['virality_score',"virality_score_pred"]]


id                                                       6881121438632267010
secretID                                                 6881121438632267010
text                       #dogsoftiktok #dogs #puppy #fyp #outfit I alwa...
createTime                                                        1602135936
authorMeta.id                                            6613681203701334017
authorMeta.secUid          MS4wLjABAAAAQBjaJevALeQknyP6qcYlpucoCUbDuQwd4u...
authorMeta.name                                                   minging888
authorMeta.nickName                                                 黃吉吉的無聊日子
authorMeta.verified                                                    False
authorMeta.signature            在臉書上有同名粉專＂黃吉吉的無聊日子＂唷\n更多影片都在FB粉專 喜歡的可以按讚❤️❤️
authorMeta.avatar          https://p16-sign-va.tiktokcdn.com/tos-useast2a...
authorMeta.following                                                       9
authorMeta.fans                                                      5100000

In [ ]:
rf_model = RandomForestRegressor(
)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(mean_squared_error(y_pred, y_test))
print(r2_score(y_pred, y_test))

3315479681432636.0
0.20471164953044596


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
compare_df = virality_df.loc[y_test.index]
compare_df['virality_score_pred'] = y_pred
compare_df.loc[:, add_cols+sub_cols+['virality_score',"virality_score_pred"]]

,diggCount,shareCount,playCount,commentCount,authorMeta.fans,virality_score,virality_score_pred
83,15900000,483200,100000000,257300,10700000,105940500.0,106055446.0
53,17600000,1200000,146500000,98600,5100000,160298600.0,159266846.0
70,16400000,100200,55200000,55600,1500000,70255800.0,83586279.0
45,18500000,365700,160600000,100600,3800000,175766300.0,177071104.0
44,18700000,189200,140700000,164100,8800000,150953300.0,149745839.0
39,19000000,223400,177600000,72000,13500000,183395400.0,180165201.0
22,21700000,1100000,100700000,270900,2900000,120870900.0,121548370.0
80,16000000,39000,112200000,121000,12700000,115660000.0,115424151.0
10,26200000,723700,212400000,291400,2000000,237615100.0,240138695.0
0,56300000,1900000,684100000,2100000,88800000,655600000.0,325474383.0


## Classification (Viral or not)
A video that is viral will be defined as having a virality score > 2.00.

In [ ]:
X = virality_df.drop(columns=["diggCount", 
                     'shareCount',
                     'playCount',
                     'commentCount',
                     'authorMeta.verified',
                     "authorMeta.following",
                     'authorMeta.fans',
                     'authorMeta.heart',
                     'authorMeta.video',
                     'authorMeta.digg'])
y = virality_df[['virality_score']] > 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [ ]:
X.iloc[:, :20].info()

In [ ]:
rf_model = RandomForestClassifier(
)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) #  confusion matrix: 
                                        # [[TN, FP
                                        #   FN, TP]]
                                  